In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('amazon sales project').getOrCreate() #Cari tau ini apa/ngaipain

24/11/22 10:19:09 WARN Utils: Your hostname, ubuntu-icanooo resolves to a loopback address: 127.0.1.1; using 192.168.100.178 instead (on interface wlp3s0)
24/11/22 10:19:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/22 10:19:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
file_path = 'assets/amazon_sales.csv'

df = spark.read.csv(file_path, header=True, inferSchema=True)

In [4]:
df.show()

+-----+-------------------+--------+--------------------+----------+--------------+------------------+--------+-------------------+-------------+----+----------+--------------+---+--------+------+-----------+--------------+----------------+------------+--------------------+-----+------------+-----------+
|index|           Order ID|    Date|              Status|Fulfilment|Sales Channel |ship-service-level|   Style|                SKU|     Category|Size|      ASIN|Courier Status|Qty|currency|Amount|  ship-city|    ship-state|ship-postal-code|ship-country|       promotion-ids|  B2B|fulfilled-by|Unnamed: 22|
+-----+-------------------+--------+--------------------+----------+--------------+------------------+--------+-------------------+-------------+----+----------+--------------+---+--------+------+-----------+--------------+----------------+------------+--------------------+-----+------------+-----------+
|    0|405-8078784-5731545|04-30-22|           Cancelled|  Merchant|     Amazon.in

In [ ]:
from pyspark.sql.functions import to_date

df = df.withColumn("Date", to_date("Date", "MM-dd-yy"))


In [ ]:
# Creating cancceled table

df_cancelled = df[df['Status'] == 'Cancelled']

In [10]:
# Removing cancelled rows

df = df[df['Status'] != 'Cancelled']

In [12]:
df.select("currency").distinct().show()

+--------+
|currency|
+--------+
|     INR|
|    NULL|
+--------+



In [22]:
from currency_converter import CurrencyConverter
from pyspark.sql.functions import udf, lit
from pyspark.sql.types import DoubleType

def changeCurrency(amount, inCurrency, outCurrency):
    try:
        c = CurrencyConverter()

        amount_in_usd = c.convert(amount, inCurrency, outCurrency)
        return amount_in_usd
    except:
        return None

changeCurrency_udf = udf(changeCurrency, DoubleType())

df = df.withColumn("Amount_in_usd", changeCurrency_udf(df["Amount"], df["currency"], lit("USD")))

In [28]:
df = df.drop("Unnamed: 22")

In [29]:
df.show()

+-----+-------------------+----------+--------------------+----------+--------------+------------------+--------+---------------+--------+----+----------+--------------+---+--------+------+---------------+--------------+----------------+------------+--------------------+-----+------------+------------------+
|index|           Order ID|      Date|              Status|Fulfilment|Sales Channel |ship-service-level|   Style|            SKU|Category|Size|      ASIN|Courier Status|Qty|currency|Amount|      ship-city|    ship-state|ship-postal-code|ship-country|       promotion-ids|  B2B|fulfilled-by|     Amount_in_usd|
+-----+-------------------+----------+--------------------+----------+--------------+------------------+--------+---------------+--------+----+----------+--------------+---+--------+------+---------------+--------------+----------------+------------+--------------------+-----+------------+------------------+
|    1|171-9198151-1101146|2022-04-30|Shipped - Deliver...|  Merchant|

In [30]:
df.printSchema()

root
 |-- index: integer (nullable = true)
 |-- Order ID: string (nullable = true)
 |-- Date: date (nullable = true)
 |-- Status: string (nullable = true)
 |-- Fulfilment: string (nullable = true)
 |-- Sales Channel : string (nullable = true)
 |-- ship-service-level: string (nullable = true)
 |-- Style: string (nullable = true)
 |-- SKU: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- ASIN: string (nullable = true)
 |-- Courier Status: string (nullable = true)
 |-- Qty: integer (nullable = true)
 |-- currency: string (nullable = true)
 |-- Amount: double (nullable = true)
 |-- ship-city: string (nullable = true)
 |-- ship-state: string (nullable = true)
 |-- ship-postal-code: double (nullable = true)
 |-- ship-country: string (nullable = true)
 |-- promotion-ids: string (nullable = true)
 |-- B2B: boolean (nullable = true)
 |-- fulfilled-by: string (nullable = true)
 |-- Amount_in_usd: double (nullable = true)

